In [1]:
import scglue.models.glue as glue
import scglue.models.sc as sc
from typing import List, Mapping, Optional, Tuple, Union
import torch
import torch.distributions as D
import torch.nn.functional as F
from scglue.num import EPS
import scglue.models.data as d
import scglue.models.scglue as sc
import anndata as ad
import networkx as nx
import scglue


# generate data 
rna = ad.read_h5ad("rna-pp2.h5ad")

atac = ad.read_h5ad("atac-pp2.h5ad")
guidance = nx.read_graphml("guidance2.graphml.gz")

scglue.models.configure_dataset(
    rna, "NB", use_highly_variable=True,
    use_layer="counts", use_rep="X_pca"
)

scglue.models.configure_dataset(
    atac, "NB", use_highly_variable=True,
    use_rep="X_lsi"
)
adatas={"rna": rna, "atac": atac}

"""
#to undertsand what is going on in 
glue = scglue.models.fit_SCGLUE(
    {"rna": rna, "atac": atac}, guidance_hvf,
    fit_kws={"directory": "glue"}
)

"""

'\nto undertsand what is going on in \nglue = scglue.models.fit_SCGLUE(\n    {"rna": rna, "atac": atac}, guidance_hvf,\n    fit_kws={"directory": "glue"}\n)\n\n'

In [2]:
#define vertices 
import pandas as pd 
import numpy as np
vertices=pd.Index(sorted(guidance.nodes))

In [3]:
pretrain_init_kws={}
pretrain_init_kws.update({"shared_batches": False})


pretrain = scglue.models.scglue.SCGLUEModel(adatas,sorted(guidance.nodes),**pretrain_init_kws)
print(pretrain.net)


[INFO] autodevice: Using CPU as computation device.
SCGLUE(
  (g2v): GraphEncoder(
    (conv): GraphConv()
    (loc): Linear(in_features=50, out_features=50, bias=True)
    (std_lin): Linear(in_features=50, out_features=50, bias=True)
  )
  (v2g): GraphDecoder()
  (x2u): ModuleDict(
    (rna): NBDataEncoder(
      (linear_0): Linear(in_features=100, out_features=256, bias=True)
      (act_0): LeakyReLU(negative_slope=0.2)
      (bn_0): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout_0): Dropout(p=0.2, inplace=False)
      (linear_1): Linear(in_features=256, out_features=256, bias=True)
      (act_1): LeakyReLU(negative_slope=0.2)
      (bn_1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout_1): Dropout(p=0.2, inplace=False)
      (loc): Linear(in_features=256, out_features=50, bias=True)
      (std_lin): Linear(in_features=256, out_features=50, bias=True)
    )
    (atac): NBDataEncoder(
   

In [ ]:
#fit step: very slow
pretrain.fit(adatas,guidance,**pretrain_fit_kws)

[INFO] check_graph: Checking variable coverage...
[INFO] check_graph: Checking edge attributes...
[INFO] check_graph: Checking self-loops...
[INFO] check_graph: Checking graph symmetry...
[INFO] check_graph: All checks passed!
[INFO] SCGLUEModel: Setting `graph_batch_size` = 202905
[INFO] SCGLUEModel: Setting `max_epochs` = 335
[INFO] SCGLUEModel: Setting `patience` = 28
[INFO] SCGLUEModel: Setting `reduce_lr_patience` = 14
[INFO] SCGLUETrainer: Using training directory: "glue/pretrain"
[INFO] SCGLUETrainer: [Epoch 10] train={'g_nll': 0.417, 'g_kl': 0.0, 'g_elbo': 0.417, 'x_rna_nll': 0.202, 'x_rna_kl': 0.011, 'x_rna_elbo': 0.213, 'x_atac_nll': 0.034, 'x_atac_kl': 0.0, 'x_atac_elbo': 0.035, 'dsc_loss': 0.692, 'vae_loss': 0.265, 'gen_loss': 0.23}, val={'g_nll': 0.417, 'g_kl': 0.0, 'g_elbo': 0.417, 'x_rna_nll': 0.209, 'x_rna_kl': 0.01, 'x_rna_elbo': 0.219, 'x_atac_nll': 0.035, 'x_atac_kl': 0.0, 'x_atac_elbo': 0.035, 'dsc_loss': 0.693, 'vae_loss': 0.27, 'gen_loss': 0.236}, 76.1s elapsed
[I

In [4]:
#define modalities
modalities2=pretrain.modalities['rna']

In [5]:
#define data encoder 
encoder=pretrain.net.x2u['rna']

In [6]:
#define pretrain_fit_kws 
import os
pretrain_fit_kws={'directory': 'glue'}
pretrain_fit_kws.update({"align_burnin": np.inf, "safe_burnin": False})

print(pretrain_fit_kws)

if "directory" in pretrain_fit_kws:
    pretrain_fit_kws["directory"] = os.path.join(pretrain_fit_kws["directory"], "pretrain")
    
pretrain_fit_kws

{'directory': 'glue', 'align_burnin': inf, 'safe_burnin': False}


{'directory': 'glue/pretrain', 'align_burnin': inf, 'safe_burnin': False}

In [7]:
#define compile_kws as empty 
compile_kws={}
pretrain.compile(**compile_kws)

In [8]:
# pretrain.fit
# pretrain.fit(adatas, graph, **pretrain_fit_kws)
#SCGLUEModel.fit, Model.fit, GLUETrainer.fit 

if "directory" in pretrain_fit_kws:
    pretrain.save(os.path.join(pretrain_fit_kws["directory"], "pretrain.dill"))
# save pretrain model

In [9]:
for k, adata in adatas.items():
    print(adata.obsm)

AxisArrays with keys: X_pca, X_umap
AxisArrays with keys: X_lsi


In [10]:
from scglue.utils import config, logged
config.TMP_PREFIX

'GLUETMP'

In [11]:
for k, adata in adatas.items():
    adata.obsm[f"X_{config.TMP_PREFIX}"] = pretrain.encode_data(k, adata)
    print(adata.obsm)

AxisArrays with keys: X_pca, X_umap, X_GLUETMP
AxisArrays with keys: X_lsi, X_GLUETMP


In [12]:
from scglue.data import estimate_balancing_weight
use_batch = None
balance_kws={}

estimate_balancing_weight(
        *adatas.values(), use_rep=f"X_{config.TMP_PREFIX}", use_batch=None,
        key_added="balancing_weight", **balance_kws
    )
#--------end of pretrain---------

[INFO] estimate_balancing_weight: Clustering cells...
[INFO] estimate_balancing_weight: Matching clusters...
[INFO] estimate_balancing_weight: Matching array shape = (12, 14)...
[INFO] estimate_balancing_weight: Estimating balancing weight...


/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/scglue/data.py:505: RuntimeWarning: invalid value encountered in true_divide
  balancing /= balancing.sum() / balancing.size
/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/scglue/data.py:505: RuntimeWarning: invalid value encountered in true_divide
  balancing /= balancing.sum() / balancing.size


In [13]:
#data_config: configure_dataset is called two times before training

print(config.ANNDATA_KEY)
for k,adata in adatas.items():
    print(adata.uns['__scglue__'])

__scglue__
{'prob_model': 'NB', 'use_highly_variable': True, 'features': ['1110006O24Rik', '1110060G06Rik', '1700001J03Rik', '1700027A07Rik', '1700039E22Rik', '1700047M11Rik', '1700063H06Rik', '1700096K18Rik', '1700120B22Rik', '1700121L03Rik', '1810010H24Rik', '1810019D21Rik', '2010013B24Rik', '2210414B05Rik', '2410004I01Rik', '2510046G10Rik', '2810408I11Rik', '4930417O22Rik', '4930426D05Rik', '4930579K19Rik', '4930597A21Rik', '4932702P03Rik', '4933405L10Rik', '5033403F01Rik', '5330416C01Rik', '5730435O14Rik', '6530403H02Rik', 'A230052G05Rik', 'A730036I17Rik', 'A830018L16Rik', 'A830029E22Rik', 'A930005G22Rik', 'AV099323', 'Abtb2', 'Ace', 'Ackr2', 'Acp5', 'Acsbg1', 'Adap2', 'Adarb2', 'Adgrf2', 'Adrb2', 'Agmat', 'Alcam', 'Aldh1a3', 'Aldoc', 'Alk', 'Ankfn1', 'Aox1', 'Ap1m2', 'Apobr', 'Apoe', 'Appl2', 'Arc', 'Areg', 'Arhgdib', 'Arl9', 'Arx', 'Asgr1', 'Aspa', 'Astn2', 'Atp10a', 'Atp13a5', 'Atp1a2', 'B930094E09Rik', 'BC006965', 'Baiap3', 'Baz1a', 'Bcan', 'Bcas1', 'Bcdin3d', 'Bco2', 'Bdnf', '

In [15]:
#define modalities: SCGLUEModel__init__ firstly used 
import copy
modalities, idx, x2u, u2x, all_ct = {}, {}, {}, {}, set()
import scglue.models.sc as sc

for k, adata in adatas.items():
    print(k,adata)
    data_config = copy.deepcopy(adata.uns[config.ANNDATA_KEY])
    modalities[k] = data_config
    idx[k]=vertices.get_indexer(data_config["features"]).astype(np.int64)
    idx[k]=torch.as_tensor(idx[k])
    
    x2u[k] = sc.NBDataEncoder(
                data_config["rep_dim"] or len(data_config["features"]), 50,
                h_depth=2, h_dim=256, dropout=0.2
            )
    data_config["batches"] = pd.Index([]) if data_config["batches"] is None \
                else pd.Index(data_config["batches"])
#     x2u[k] = sc.NBDataDecoder(
#                 data_config["rep_dim"] or len(data_config["features"]), latent_dim,
#                 h_depth=h_depth, h_dim=h_dim, dropout=dropout
#             )

rna AnnData object with n_obs × n_vars = 1000 × 28930
    obs: 'domain', 'cell_type', 'balancing_weight'
    var: 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'mean', 'std', 'chrom', 'chromStart', 'chromEnd', 'name', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb', 'blockCount', 'blockSizes', 'blockStarts', 'gene_id', 'gene_type', 'mgi_id', 'havana_gene', 'tag'
    uns: 'hvg', 'log1p', 'neighbors', 'pca', 'umap', '__scglue__'
    obsm: 'X_pca', 'X_umap', 'X_GLUETMP'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'
atac AnnData object with n_obs × n_vars = 9190 × 241757
    obs: 'domain', 'cell_type', 'balancing_weight'
    var: 'chrom', 'chromStart', 'chromEnd', 'highly_variable'
    uns: '__scglue__'
    obsm: 'X_lsi', 'X_GLUETMP'


In [16]:
#define dataloader and Anndata
from scglue.models.data import AnnDataset, ArrayDataset, DataLoader, GraphDataset

anndata = AnnDataset(
            [adatas[key] for key in ['rna']],
            [modalities[key] for key in ['rna']],
            mode="train"
        )

data_loader = DataLoader( #three elements: x, xrep, _
            anndata, batch_size=128,
            shuffle=False, drop_last=False
        )



In [17]:
#define configs2
data_config2=anndata._data_configs
 


In [ ]:
# def freeze_u(freeze_u: bool): require_grad
#         for item in chain(self.net.x2u.parameters(), self.net.du.parameters()):
#             item.requires_grad_(not self._freeze_u)


In [ ]:
# def compile(  # pylint: disable=arguments-differ
#             self, lam_data: float = 1.0,
#             lam_kl: float = 1.0,
#             lam_graph: float = 0.02,
#             lam_align: float = 0.05,
#             lam_sup: float = 0.02,
#             normalize_u: bool = False,
#             modality_weight: Optional[Mapping[str, float]] = None,
#             lr: float = 2e-3, **kwargs
#     ) -> None:
# in scglue.py : why do we need modalities weight
#  modality_weight: Relative modality weight (indexed by modality name)


In [18]:
# SCGLUETrainer---- self._freeze_u
#freeze_u = False

#format_data 

#compute_losses 
lam_data=1
lam_kl=1.0
lam_graph=0.02
lam_align=0.05
lam_sup=0.02
modality_weight={'rna': 1.0, 'atac': 1.0}
normalize_u=False
lr=0.002
optim=str("RMSprop")
#train_step 
from scglue.models.scglue import SCGLUETrainer
import torch.optim as optim 

'''
net: SCGLUE, lam_data: float = None, lam_kl: float = None,
            lam_graph: float = None, lam_align: float = None,
            lam_sup: float = None, normalize_u: bool = None,
            modality_weight: Mapping[str, float] = None,
            optim: str = None, lr: float = None,'''

trainer=SCGLUETrainer(pretrain.net,lam_data,lam_kl,lam_graph,lam_align,lam_sup,normalize_u,modality_weight,"RMSprop",lr)

trainer.format_data(anndata)



TypeError: unsupported operand type(s) for *: 'slice' and 'int'

In [ ]:
class Base:
    def __init__(self):
        self.a = "GeeksforGeeks"
        self.__c = "Geeks+++forGeeks"# encapsulated variable
    
    def test(self,name):
        print(f'base is used {name}')
    
    def __test2(self):
        print('encapsulated method')
  
# Creating a derived class
class Derived(Base):
    def __init__(self):
  
        # Calling constructor of
        # Base class
        Base.__init__(self)
        print("Calling private member of base class: ")
        print(self.a)
        
obj1 = Derived()
obj1.test('world')
# obj1.__test2() private method 


In [27]:
data_config=adata.uns['__scglue__']# adata.uns[config.ANNDATA_KEY] = data_config, change made by configure_dataset

for k, adata in adatas.items():
    print(adata)

AnnData object with n_obs × n_vars = 1000 × 28930
    obs: 'domain', 'cell_type', 'balancing_weight'
    var: 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'mean', 'std', 'chrom', 'chromStart', 'chromEnd', 'name', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb', 'blockCount', 'blockSizes', 'blockStarts', 'gene_id', 'gene_type', 'mgi_id', 'havana_gene', 'tag'
    uns: 'hvg', 'log1p', 'neighbors', 'pca', 'umap', '__scglue__'
    obsm: 'X_pca', 'X_umap', 'X_GLUETMP'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'
AnnData object with n_obs × n_vars = 9190 × 241757
    obs: 'domain', 'cell_type', 'balancing_weight'
    var: 'chrom', 'chromStart', 'chromEnd', 'highly_variable'
    uns: '__scglue__'
    obsm: 'X_lsi', 'X_GLUETMP'


In [75]:
#to get extracted data 
xuid, (x, xrep, xbch, xlbl, xdwt)=anndata._extract_data_train(data_config2)
extracted_data=anndata._extract_data_train(data_config2)
extracted_data

([Index(['a697c4dfc399498588b58e240e7dd83c', '9221e84eef58452a83af3e3209f1a5e5',
         '2d22b8c343d74f81acde48ed879dbab2', '864821547199423ba63e65fee1c9b02c',
         'fe3114d8098942d197d075912d10949a', 'c0331005fd4c4518972c810d0f876e8f',
         'd5ebaff04dc640bfb71c719d765ace69', 'dbe8641379be43fdb6a9d8bb9b80fd45',
         '865d8f2bcd0d4ff58b963281d6becc83', 'b03663d5eebc497d8c46130c17fa18df',
         ...
         'b79a27da0c1044e08195970a5aa5efcf', 'e37b915011f447d49f3e5ec5e626f297',
         '05235e6cdcd941f28e8ba8b38aa4fd09', '877382350c5f48f1a5e73d9eaf2168ad',
         '42666e8270f34824b7cb37f00a0fcdaa', '6476d65d66bd4ac5af45121d0afab244',
         '969f442efe604552aa7b453f985a0399', 'ec5bb8b5129644669198f5ab69d9768f',
         '800f4bcc1e5a45bdb7441b6f2146f549', 'b63de13b4ff944818cde987aa6bebd65'],
        dtype='object', length=1000)],
 ([<1000x800 sparse matrix of type '<class 'numpy.float32'>'
   	with 52349 stored elements in Compressed Sparse Row format>],
  [array([

In [24]:
xuid

[Index(['7b301024c6474d67a955fea904e12e2b', '59c4f3c2c62348329d07968fb715d538',
        'a4830386e9ff4b5d9c884b9ce878767a', 'c4a4f5e4d52b4b748021676fd19d285c',
        'e91348d815804e2192c39cc9d00afd6c', 'a8043519686047bca2b4b25c61fad734',
        '122eb675933041c4a8ddb9575b3c3296', 'd8873655527247aa99b4eac866b4f01e',
        'd34fccd9f3c5482192c93ab9c4647b70', 'a201cd9587ca4021bf1abd520d03cc26',
        ...
        '816a3b1d09e948a1b9f7176e8129446c', 'e8320504faa247b395ec0cf6d07eb79f',
        '1e83f406d5fc4c01a2dba0e14f7147a0', 'f87239278cf44fecac453c37e76eaea5',
        'aa9498c5afd047518428344496fd1734', '33bceac06a97488194bb865aae53b1be',
        '885c61970fb846c9909aed5a8290d179', 'e731566c67d04bb1a75b40ec5e3072f9',
        'fd2b1dbe216045eb85efe686d3668173', '1fd8227c9ed3491c9fe50078aa350b78'],
       dtype='object', length=1000)]

In [25]:
xdwt

[array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 

In [26]:
xlbl

[array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, 

In [29]:
len(anndata.extracted_data)

5

In [30]:
anndata.view_idx

array(['c986b0127bab41958f116b92501c6c1d',
       'f042811d0c6a418286f9d95f1a60a4a4',
       '89f6951191e74eaa9e47e116c930ddc9',
       'a1f498bc7bd747618c7a7d878b01696e',
       '1de3f222ef9d443c9d572b0a273e2338',
       '09695687d7a74b64bba8e590a9be5fca',
       '81f804c2635b4fd49eed61c23a96b53e',
       '763f60b3b59a4cee94592e70a7c8e0ee',
       'b4514cff141545c79d7955dd32bec885',
       '49e9393adf084e8ab28701a4ffdeb4ba',
       'f7e849e2bb7a45e5b81d28a0b0040325',
       'f20135133a5442c388a0b8db6760074c',
       '3890212274fc42ad93338199b67f3f29',
       '557be1c6daa943758eb46121244fead6',
       'e2d97caee28449ddbf5f5ce9e03a39fe',
       '6a3c75f6c4314ad7a2e00df382cbc986',
       '576731eb77a7487aab3951d91c20fda8',
       '3fefb35f47bb49da8e7ae32a1838923e',
       '3e2133457c3740529b5dfa59f0e896c9',
       'c4a93e67ab7d4f2c998754ad7f56d223',
       '0cbc7005649d438aa01236b7ec8b86d8',
       '47ed2c780eea440cb031cbb105d55a87',
       'c4ad98de0d6248ec9371dadfdd62a214',
       '495

In [34]:
len(anndata.__getitem__(0))

6

In [76]:
from scglue.models.data import GraphDataset 
gdata=GraphDataset(guidance,vertices)

In [77]:
eidx, ewt, esgn = gdata.graph2triplet(guidance,vertices)

In [ ]:
#input of forma_data in scglue.py/ list of tensors:  x, xrep, xbch, xlbl, xdwt, pmsk, (eidx, ewt, esgn)
#extracted data:xuid, (x, xrep, xbch, xlbl, xdwt)
#from graph: eidx, ewt, esgn 
#what is pmsk?

In [ ]:
#static method: class_name.function_name() no need to create class object 


In [78]:
pmsk=anndata.shuffle_pmsk #you can get shuffle_pmsk in data.AnnDataset._get_idx_pmsk

In [79]:
data_list=[]
data_list.append(x)#1
data_list.append(xrep)#2
data_list.append(xbch)#3
data_list.append(xlbl)#4
data_list.append(xdwt)#5
data_list.append(pmsk)#6
data_list.append(eidx)#7
data_list.append(ewt)#8
data_list.append(esgn)#9

In [80]:

def format_data(x, xrep, xbch, xlbl, xdwt, pmsk, eidx, ewt, esgn):
    """
        Format data tensors

        Note
        ----
        The data dataset should contain data arrays for each modality,
        followed by alternative input arrays for each modality,
        in the same order as modality keys of the network.
        """
#         device = self.net.device
#         keys = self.net.keys
#         K = len(keys)
#         x, xrep, xbch, xlbl, xdwt, pmsk, (eidx, ewt, esgn) = \
#             data[0:K], data[K:2*K], data[2*K:3*K], data[3*K:4*K], data[4*K:5*K], \
#             data[5*K], data[5*K+1:]
    keys=['rna']
    for i,k in enumerate(keys):
        print(i,k)
        
    x = {
            k: x[i]
            for i, k in enumerate(keys)
        }

    xrep = {
            k: xrep[i]
            for i, k in enumerate(keys)
        }

    xbch = {
            k: xbch[i]
            for i, k in enumerate(keys)
        }

    xlbl = {
            k: xlbl[i]
            for i, k in enumerate(keys)
        }

    xdwt = {
            k: xdwt[i]
            for i, k in enumerate(keys)
        }

    xflag = {
            k: torch.as_tensor(
                i
            ).expand(x[k].shape[0])
            for i, k in enumerate(keys)
        }
        
    return x, xrep, xbch, xlbl, xdwt, xflag, pmsk, eidx, ewt, esgn

In [84]:
DataTensor=format_data(x, xrep, xbch, xlbl, xdwt, pmsk, eidx, ewt, esgn)
format_x=DataTensor[0]
format_xrep=DataTensor[1]
format_xbch=DataTensor[2]
format_xlbl=DataTensor[3]
format_xdwt=DataTensor[4]
format_xflag=DataTensor[5]
format_pmsk=DataTensor[6]
format_eidx=DataTensor[7]
format_ewt=DataTensor[8]
format_esgn=DataTensor[9]

0 rna


In [86]:
format_xflag # what is xflag? and Why it is all zeros?

{'rna': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 